In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
# 모델 생성 함수
def build_model(X) :
    
    # Layer1 / ImgIn shape = (?, 64, 64, 3)
    # kernel_size = [3, 3], filters = 32
    W1 = tf.Variable(tf.random_normal([3, 3, 3, 32], stddev = 0.01))
    L1 = tf.nn.conv2d(X, W1, strides = [1, 1, 1, 1], padding = 'SAME')
    #    Conv     -> (?, 64, 64, 32)
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    #    Pool     -> (?, 32, 32, 32)
    L1 = tf.nn.dropout(L1, keep_prob = keep_prob)
    
    # Layer2 / ImgIn shape = (?, 32, 32, 32)
    # kernel_size = [3, 3], filters = 64 
    W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
    L2 = tf.nn.conv2d(L1, W2, strides = [1, 1, 1, 1], padding = 'SAME')
    #    Conv     -> (?, 32, 32, 64)
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    #    Pool     -> (?, 16, 16, 64)
    L2 = tf.nn.dropout(L2, keep_prob = keep_prob)
    
    # Layer3 / ImgIn shape = (?, 16, 16, 64)
    # kernel_size = [3, 3], filters = 128 
    W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev = 0.01))
    L3 = tf.nn.conv2d(L2, W3, strides = [1, 1, 1, 1], padding = 'SAME')
    #    Conv     -> (?, 16, 16, 128)
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    #    Pool     -> (?, 8, 8, 128)
    L3 = tf.nn.dropout(L3, keep_prob = keep_prob)
    L3_flat = tf.reshape(L3, [-1, 128 * 8 * 8])
    
    # Layer4 / FC 8x8x128 inputs -> 625 outputs
    W4 = tf.get_variable("W4", shape = [128 * 8 * 8, 625], initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([625]))
    L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob = keep_prob)
    
    # Layer5 / Final FC 625 inputs -> (N_CLASSES) outputs
    W5 = tf.get_variable("W5", shape = [625, N_CLASSES], initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([N_CLASSES]))
    logits = tf.matmul(L4, W5) + b5
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

In [3]:
LABELS = []
for folder in glob.glob('./data/*') :
    label = folder.split('/')[-1]
    # label = label.split('\\')[-1]
    # print(label)
    LABELS.append(label)
print(LABELS)

['No_overrun', 'NO_U-turn', '100', 'NO_left', 'Height', 'NO_entry', 'No_twowheel', 'NO_parking_stop', 'Weight', 'No_walk', '50', '60', '40', 'Default', 'NO_truck', 'NO_right', 'Other_Circle', 'No_straight', '30']


In [4]:
# 클래스 개수
N_CLASSES = len(LABELS)
# print(N_CLASSES)
# input image size
RESIZED_IMAGE = (64, 64)

In [5]:
# dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# input placeholders
X = tf.placeholder(tf.float32, [None, RESIZED_IMAGE[0], RESIZED_IMAGE[1], 3])
y = tf.placeholder(tf.float32, [None, N_CLASSES])

In [6]:
y_pred, logits = build_model(X)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [7]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
CHECK_POINT_DIR = './model.ckpt'
if not os.path.exists(CHECK_POINT_DIR) :
    os.makedirs(CHECK_POINT_DIR)

In [9]:
checkpoint = tf.train.get_checkpoint_state(CHECK_POINT_DIR)
if checkpoint :
    print(checkpoint)
    print(checkpoint.model_checkpoint_path)

model_checkpoint_path: "./model.ckpt/model"
all_model_checkpoint_paths: "./model.ckpt/model"

./model.ckpt/model


In [10]:
 print(checkpoint.model_checkpoint_path)

./model.ckpt/model


In [11]:
# initialize
config = ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [13]:
saver = tf.train.Saver()

In [13]:
# saver = tf.train.import_meta_graph(checkpoint.model_checkpoint_path + '.meta')

In [14]:
sess.run(tf.global_variables_initializer())

In [15]:
# load a trained model
if checkpoint and checkpoint.model_checkpoint_path :
    try : 
        # saver = tf.train.import_meta_graph(checkpoint.model_checkpoint_path + '.meta')
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print('Successfully loaded : ', checkpoint.model_checkpoint_path)
    except :
        print("Error on loading old network weights")
else :
    print("Could not find old network weights")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model.ckpt/model
Successfully loaded :  ./model.ckpt/model


In [16]:
import cv2
frame = cv2.imread('./test/no_uturn.jpg')
frame = cv2.resize(frame, (RESIZED_IMAGE[0], RESIZED_IMAGE[1]))
frame = np.expand_dims(frame, axis=0)
frame.shape

(1, 64, 64, 3)

In [17]:
pred = sess.run(tf.argmax(y_pred, axis = 1), feed_dict = {X : frame, keep_prob : 1})

In [18]:
IDX_LABEL = {}
for i, label in zip(range(0, N_CLASSES), LABELS) :
    IDX_LABEL[i] = label
print(IDX_LABEL)

{0: 'No_overrun', 1: 'NO_U-turn', 2: '100', 3: 'NO_left', 4: 'Height', 5: 'NO_entry', 6: 'No_twowheel', 7: 'NO_parking_stop', 8: 'Weight', 9: 'No_walk', 10: '50', 11: '60', 12: '40', 13: 'Default', 14: 'NO_truck', 15: 'NO_right', 16: 'Other_Circle', 17: 'No_straight', 18: '30'}


In [19]:
label = IDX_LABEL[pred[0]]
print(label)

NO_U-turn
